### 이력서 파일로 면접 질문 예상 프롬프트

In [18]:
# 생성한 이력서 파일로 면접 예상 질문 생성 (LangChain)

# 환경 설정 및 라이브러리 임포트
import os
from dotenv import load_dotenv
from typing import List, Dict

# LangChain 패키지
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.output_parsers import JsonOutputParser

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY가 설정되지 않았습니다. .env 파일을 확인해주세요.")

print(f"OPENAI_API_KEY: {OPENAI_API_KEY[:2]}")


OPENAI_API_KEY: sk


In [19]:
# 이력서 로딩 및 벡터 저장소 생성 
def load_resume_to_vector_store(file_path: str, chunk_size: int = 1000, chunk_overlap: int = 100):
    """
    이력서 파일을 로드하고 텍스트를 분할하여 FAISS 벡터 저장소를 생성합니다.
    """
    try:
        print(f"이력서 파일 로딩 중: {file_path}")

        # pdf나 txt 파일에 따라 문서 로드
        if file_path.lower().endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        elif file_path.lower().endswith(".txt"):
            loader = TextLoader(file_path)
        else:
            raise ValueError("지원되지 않는 파일 형식입니다. PDF 또는 TXT 파일만 업로드해주세요.")

        documents = loader.load()

        if not documents:
            raise ValueError("파일에서 텍스트를 추출할 수 없습니다. 파일 내용을 확인해주세요.")

        print(f"총 {len(documents)}개 문서/페이지 로드됨")

        # 텍스트 분할
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=["\n\n", "\n", ".", " ", ""]
        )
        splits = text_splitter.split_documents(documents)
        print(f"총 {len(splits)}개 청크로 분할됨")

        # 임베딩 모델 생성
        embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model="text-embedding-3-small")

        # FAISS 벡터 저장소 생성
        print("FAISS 벡터 저장소 생성 중...")
        vectorstore = FAISS.from_documents(
            documents=splits,
            embedding=embeddings
        )

        print("벡터 저장소 생성 완료!")
        return vectorstore

    except Exception as e:
        print(f"이력서 로딩 및 벡터 저장소 생성 중 오류 발생: {str(e)}")
        raise e

In [20]:
# 면접 예상 질문 생성 
def generate_interview_questions(
    vectorstore,
    company_name: str = "",
    desired_job_role: str = "", # 새로 추가된 파라미터: 희망 직무
    interview_type: str = "general", # 'general', 'technical', 'behavioral'
    temperature: float = 0.7
) -> Dict:
    """
    주어진 벡터 저장소 (이력서 내용)와 파라미터를 기반으로 면접 예상 질문을 생성합니다.
    생성된 질문은 JSON 형식으로 반환됩니다.
    """
    try:
        # 검색기 설정 (이력서 내용 검색)
        retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 5} # 관련성 높은 5개 청크 검색
        )

        # 질문 유형에 따른 추가 지침
        type_instructions = {
            "general": "지원자의 전반적인 경험, 역량, 회사 적합성 등을 평가할 수 있는 종합적인 질문을 생성해주세요.",
            "technical": "지원자의 이력서에 언급된 기술 스택과 프로젝트 경험을 바탕으로 심도 있는 기술 면접 질문을 생성해주세요.",
            "behavioral": "지원자의 행동 양식, 문제 해결 능력, 팀워크, 리더십 등을 평가할 수 있는 인성 및 행동 기반 면접 질문을 생성해주세요."
        }
        instruction = type_instructions.get(interview_type, type_instructions["general"])

        # 회사 이름이 제공된 경우 추가 지침
        company_instruction = f"면접 볼 회사가 '{company_name}'이라면, 해당 회사의 비전, 제품, 문화와 연관된 질문을 1~2개 포함해주세요." if company_name else ""
        
        # 희망 직무가 제공된 경우 추가 지침
        job_role_instruction = ""
        if desired_job_role:
            job_role_instruction = f"지원자가 희망하는 직무는 '{desired_job_role}'입니다. 이 직무에 필요한 역량, 경험, 그리고 직무 관련 시나리오에 대한 질문을 1~2개 포함해주세요."
            if interview_type == "technical":
                job_role_instruction += f" 특히 '{desired_job_role}' 직무의 핵심 기술 스택과 관련된 심층 기술 질문을 포함해주세요."
            elif interview_type == "behavioral":
                job_role_instruction += f" 특히 '{desired_job_role}' 직무에서 발생할 수 있는 상황에 대한 행동 기반 질문을 포함해주세요."

        # 프롬프트 템플릿 정의
        template = f"""
        당신은 면접관이며, 제공된 이력서 내용을 바탕으로 지원자에게 면접 예상 질문을 생성해야 합니다.
        다음 지침에 따라 질문을 생성하고, 각 질문에 대한 간략한 답변 가이드라인도 함께 제공해주세요.
        응답은 반드시 JSON 형식으로만 반환해야 합니다.

        <이력서 내용>
        {{context}}
        </이력서 내용>

        지침:
        1.  이력서 내용을 면밀히 분석하여 지원자의 경험, 기술 스택, 강점, 약점 등을 파악하세요.
        2.  총 5~7개의 면접 질문을 생성해주세요.
        3.  질문 유형: {instruction} {company_instruction} {job_role_instruction}
        4.  각 질문에 대해 답변자가 어떤 점을 강조해야 하는지 1~2문장으로 간략한 '가이드라인'을 포함해주세요.
        5.  전반적인 답변 가이드라인을 1~2문장으로 요약하여 추가해주세요.

        응답 JSON 스키마:
        {{{{
        "questions": [
            {{{{
            "question": "질문 내용 1",
            "guidance": "답변 가이드라인 1"
            }}}},
            {{{{
            "question": "질문 내용 2",
            "guidance": "답변 가이드라인 2"
            }}}}
        ],
        "overall_guidance": "전반적인 답변 가이드라인 요약"
        }}}}

        질문: 이 이력서를 가진 지원자에게 어떤 면접 질문을 할까요?

        답변:"""

        prompt = ChatPromptTemplate.from_template(template)

        # LLM 모델 설정
        # temperature 값 -> 질문의 다양성(창의성)에 영향을 줌
        model = ChatOpenAI(
            model='gpt-4o', 
            temperature=float(temperature),
            api_key=OPENAI_API_KEY
        )

        # JSON 출력을 위한 파서 설정
        parser = JsonOutputParser()

        # RAG 체인 생성
        document_chain = create_stuff_documents_chain(model, prompt)
        rag_chain = create_retrieval_chain(retriever, document_chain)

        # 답변 생성
        response = rag_chain.invoke({'input': "면접 질문을 생성해주세요."})
        
        # LLM 응답이 JSON 문자열일 경우 파싱
        # LangChain의 create_retrieval_chain은 'answer' 키에 최종 응답을 반환
        # 그 응답은 LLM이 생성한 JSON 문자열이므로 파싱 필요
        parsed_response = parser.parse(response['answer'])
        return parsed_response

    except Exception as e:
        print(f"면접 질문 생성 중 오류 발생: {str(e)}")
        return {"error": f"면접 질문 생성 중 오류가 발생했습니다: {str(e)}"}


In [21]:
# 제공하는 PDF 파일로 예상 면접 질문 생성 - 마케터 버전
resume_file_path = "./resume/예시 파일 (마케터).pdf"

# 파일 존재 여부 확인 (오류 방지용)
if not os.path.exists(resume_file_path):
    print(f"오류: 지정된 경로에 파일이 없습니다: {resume_file_path}")
    print("파일 경로를 확인하거나, 파일을 해당 위치에 넣어주세요.")
else:
    print(f"\n--- '{resume_file_path}' 파일로 면접 예상 질문 생성 ---")
    try:
        # 1. PDF 파일 로드 및 벡터 저장소 생성
        vectorstore_marketer = load_resume_to_vector_store(resume_file_path)

        # 2. 면접 예상 질문 생성
        # 마케터 이력서이므로 'general' 또는 'behavioral' 유형이 적합할 수 있습니다.
        # 필요에 따라 company_name을 추가할 수 있습니다.
        questions_marketer = generate_interview_questions(
            vectorstore_marketer,
            company_name="ABC 마케팅", # 예시 회사명
            interview_type="general", # 일반 면접 질문
            temperature=0.7
        )

        # 3. 결과 출력
        if "error" in questions_marketer:
            print(f"오류: {questions_marketer['error']}")
        else:
            print("\n생성된 면접 질문:")
            for i, q in enumerate(questions_marketer['questions']):
                print(f"{i+1}. {q['question']}")
                print(f"   가이드라인: {q['guidance']}")
            print(f"\n전반적인 가이드라인: {questions_marketer['overall_guidance']}")

    except Exception as e:
        print(f"예상치 못한 오류 발생: {e}")


--- './resume/예시 파일 (마케터).pdf' 파일로 면접 예상 질문 생성 ---
이력서 파일 로딩 중: ./resume/예시 파일 (마케터).pdf
총 2개 문서/페이지 로드됨
총 4개 청크로 분할됨
FAISS 벡터 저장소 생성 중...
벡터 저장소 생성 완료!

생성된 면접 질문:
1. 디지털 마케팅 실전 스터디에서 얻은 가장 큰 교훈은 무엇인가요?
   가이드라인: 스터디를 통해 실전 경험을 쌓으며 배운 점과 이를 실제 업무에 어떻게 적용할 수 있는지를 설명하세요.
2. 뷰티 산업에서의 최신 트렌드를 어떻게 분석하고 콘텐츠에 반영하셨나요?
   가이드라인: 최신 트렌드를 파악하는 데 사용한 방법과 이를 콘텐츠 기획 및 제작에 어떻게 반영했는지 구체적으로 설명하세요.
3. ABC 마케팅의 제품이나 서비스에 대한 디지털 마케팅 전략을 제안해 주실 수 있나요?
   가이드라인: ABC 마케팅의 비전과 제품을 고려한 맞춤형 디지털 마케팅 전략을 제시하고, 예상되는 성과를 설명하세요.
4. 유튜브와 인스타그램 채널 운영 경험을 통해 얻은 가장 큰 성과는 무엇이며, 이 경험이 ABC 마케팅에서 어떻게 활용될 수 있다고 생각하십니까?
   가이드라인: 구독자 및 팔로워 증가, 조회수 등 구체적인 성과를 언급하고, 이러한 경험이 ABC 마케팅에서 어떻게 기여할 수 있는지 설명하세요.
5. 사회적 가치를 담은 뷰티 브랜드 콘텐츠 기획에서 주목할만한 점은 무엇이었나요?
   가이드라인: 사회적 가치를 중심으로 한 콘텐츠 기획의 중요성과 그 성공 사례를 설명하고, 이 경험이 회사의 가치와 어떻게 부합할 수 있는지 설명하세요.
6. 데이터를 기반으로 한 마케팅 성과 분석의 중요성을 체감한 경험을 공유해 주세요.
   가이드라인: 데이터 분석을 통해 얻은 인사이트와 이를 어떻게 콘텐츠 개선에 활용했는지를 구체적으로 설명하세요.

전반적인 가이드라인: 각 답변에서 구체적인 경험과 성과를 강조하고, 이를 ABC 마케팅의 목표와 어떻게 연결할 수 있는지를 설명하세요.


In [22]:
# 제공하는 PDF 파일로 예상 면접 질문 생성 - 개발자 버전
resume_file_path = "./resume/예시파일 (개발자).pdf"

# 파일 존재 여부 확인 (오류 방지용)
if not os.path.exists(resume_file_path):
    print(f"오류: 지정된 경로에 파일이 없습니다: {resume_file_path}")
    print("파일 경로를 확인하거나, 파일을 해당 위치에 넣어주세요.")
else:
    print(f"\n--- '{resume_file_path}' 파일로 면접 예상 질문 생성 ---")
    try:
        # 1. PDF 파일 로드 및 벡터 저장소 생성
        vectorstore_marketer = load_resume_to_vector_store(resume_file_path)

        # 2. 면접 예상 질문 생성
        # 필요에 따라 company_name을 추가할 수 있습니다.
        questions_marketer = generate_interview_questions(
            vectorstore_marketer,
            company_name="삼성", # 예시 회사명
            interview_type="technical", # 일반 면접 질문
            temperature=0.7
        )

        # 3. 결과 출력
        if "error" in questions_marketer:
            print(f"오류: {questions_marketer['error']}")
        else:
            print("\n생성된 면접 질문:")
            for i, q in enumerate(questions_marketer['questions']):
                print(f"{i+1}. {q['question']}")
                print(f"   가이드라인: {q['guidance']}")
            print(f"\n전반적인 가이드라인: {questions_marketer['overall_guidance']}")

    except Exception as e:
        print(f"예상치 못한 오류 발생: {e}")


--- './resume/예시파일 (개발자).pdf' 파일로 면접 예상 질문 생성 ---
이력서 파일 로딩 중: ./resume/예시파일 (개발자).pdf
총 3개 문서/페이지 로드됨
총 5개 청크로 분할됨
FAISS 벡터 저장소 생성 중...
벡터 저장소 생성 완료!

생성된 면접 질문:
1. Spring Boot와 Microservices 아키텍처를 활용하여 대규모 트래픽을 처리하는 시스템을 설계하고 최적화한 경험에 대해 설명해주세요.
   가이드라인: 트래픽 처리 개선을 위한 구체적인 전략과 성과를 설명하고, 경험을 통해 배운 점을 강조하세요.
2. Kafka 기반 이벤트 드리븐 아키텍처를 도입한 경험에 대해 설명해주시고, 이를 통해 얻은 이점과 해결한 문제는 무엇인가요?
   가이드라인: Kafka 도입으로 해결한 문제와 얻은 성과를 구체적으로 설명하고, 이벤트 드리븐 아키텍처의 장점을 강조하세요.
3. Jenkins를 활용한 CI/CD 파이프라인 구축 경험이 있는데, 이를 통해 프로젝트에 가져온 변화와 이점을 설명해주세요.
   가이드라인: CI/CD 파이프라인이 프로젝트의 효율성과 안정성에 미친 긍정적인 영향을 구체적으로 설명하세요.
4. 삼성의 글로벌 제품 개발과 관련하여, 귀하의 기술 스택 중 어떤 부분이 기여할 수 있을지 설명해주세요.
   가이드라인: 삼성의 제품 개발에 어떻게 기여할 수 있을지, 자신의 기술 역량과 경험을 연결하여 설명하세요.
5. Redis 캐싱 전략을 적용하여 성능을 향상시킨 경험에 대해 설명해주시고, 그로 인해 얻은 성과는 무엇인가요?
   가이드라인: Redis를 활용한 성능 최적화 경험과 이를 통한 성과를 구체적으로 설명하세요.
6. OpenCV와 Python을 활용한 프로젝트 경험이 있는데, 이미지 처리 관련 기술을 어떻게 활용했는지 설명해주세요.
   가이드라인: OpenCV와 Python을 사용하여 해결한 문제와 프로젝트 결과를 구체적으로 설명하세요.
7. 사내 ERP 시스템 유지보수 및 기능 개선을 진행하면

In [23]:
# 희망 직무 추가 버전
resume_file_path = "./resume/예시 파일 (마케터).pdf"

if not os.path.exists(resume_file_path):
    print(f"오류: 지정된 경로에 파일이 없습니다: {resume_file_path}")
    print("파일 경로를 확인하거나, 파일을 해당 위치에 넣어주세요.")
else:
    print(f"\n--- '{resume_file_path}' 파일로 희망 직무 '디지털 마케터' 면접 예상 질문 생성 ---")
    try:
        # 1. PDF 파일 로드 및 벡터 저장소 생성
        vectorstore_marketer = load_resume_to_vector_store(resume_file_path)

        # 2. 면접 예상 질문 생성 (희망 직무 추가)
        questions_marketer = generate_interview_questions(
            vectorstore_marketer,
            company_name="ABC 회사", # 예시 회사명
            interview_type="general", # 일반 면접 질문
            desired_job_role="디지털 마케터", # 희망 직무 입력
            temperature=0.7
        )

        # 3. 결과 출력
        if "error" in questions_marketer:
            print(f"오류: {questions_marketer['error']}")
        else:
            print("\n생성된 면접 질문:")
            for i, q in enumerate(questions_marketer['questions']):
                print(f"{i+1}. {q['question']}")
                print(f"   가이드라인: {q['guidance']}")
            print(f"\n전반적인 가이드라인: {questions_marketer['overall_guidance']}")

    except Exception as e:
        print(f"예상치 못한 오류 발생: {e}")


--- './resume/예시 파일 (마케터).pdf' 파일로 희망 직무 '디지털 마케터' 면접 예상 질문 생성 ---
이력서 파일 로딩 중: ./resume/예시 파일 (마케터).pdf
총 2개 문서/페이지 로드됨
총 4개 청크로 분할됨
FAISS 벡터 저장소 생성 중...
벡터 저장소 생성 완료!

생성된 면접 질문:
1. 마케팅 스터디와 뷰티 에디터 서포터즈 활동에서 얻은 가장 큰 배움은 무엇이었나요?
   가이드라인: 실제 경험을 통해 얻은 새로운 통찰이나 기술, 특히 디지털 마케팅과 콘텐츠 기획 분야에서의 성장을 설명하세요.
2. ABC 회사의 비전과 가치에 대해 어떻게 생각하시며, 이와 관련하여 본인이 기여할 수 있는 부분은 무엇인가요?
   가이드라인: ABC 회사의 비전과 가치에 대해 이해하고 있으며, 본인의 경험과 역량이 회사의 목표 달성에 어떻게 기여할 수 있는지를 설명하세요.
3. 디지털 마케팅 분야에서 최신 뷰티 트렌드를 어떻게 분석하고 콘텐츠에 반영하시나요?
   가이드라인: 최신 트렌드를 분석하는 방법과 이를 콘텐츠에 효과적으로 반영하는 전략을 설명하고, 이를 통해 성공한 사례를 언급하세요.
4. YouTube와 Instagram 채널을 운영하면서 직면했던 가장 큰 도전은 무엇이었고, 이를 어떻게 극복하셨나요?
   가이드라인: 구체적인 도전 사례를 설명하고, 이를 극복하기 위한 전략과 그 결과를 명확히 제시하세요.
5. ABC 회사의 제품이나 서비스 중 하나를 선택하여, 이를 홍보하기 위한 디지털 마케팅 전략을 제안해 보세요.
   가이드라인: ABC 회사의 특정 제품이나 서비스를 선택하고, 타겟 고객층 및 마케팅 목표에 맞춘 구체적인 디지털 마케팅 전략을 제시하세요.
6. 데이터 기반의 콘텐츠 성과 분석을 통해 개선한 사례를 설명해 주세요.
   가이드라인: 분석한 데이터와 그에 따른 구체적인 개선 조치를 설명하고, 이를 통해 얻은 긍정적인 결과를 강조하세요.
7. 디지털 마케터로서 본인의 가장 큰 강점은 무엇이며, 이를 통해 A

### 이력서 파일 참고하여 추천 직무, 역량 추천

In [24]:
# 추천 직무 및 필요 역량 생성

# 환경 설정 및 라이브러리 임포트
import os
from dotenv import load_dotenv
from typing import List, Dict, Optional

# LangChain 패키지
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.output_parsers import JsonOutputParser # JSON 형식으로 출력 파싱

# API KEY 확인
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY가 설정되지 않았습니다. .env 파일을 확인해주세요.")

print(f"OPENAI_API_KEY: {OPENAI_API_KEY[:2]}")

OPENAI_API_KEY: sk


In [25]:
# 이력서 로딩 및 벡터 저장소 생성 함수
def load_resume_to_vector_store(file_path: str, chunk_size: int = 1000, chunk_overlap: int = 100):
    """
    이력서 파일을 로드하고 텍스트를 분할하여 FAISS 벡터 저장소를 생성합니다.
    지원 형식: PDF, TXT
    """
    try:
        print(f"이력서 파일 로딩 중: {file_path}")

        # 파일 확장자에 따라 적절한 로더 사용
        if file_path.lower().endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        elif file_path.lower().endswith(".txt"):
            loader = TextLoader(file_path)
        else:
            raise ValueError("지원되지 않는 파일 형식입니다. PDF 또는 TXT 파일만 업로드해주세요.")

        documents = loader.load()

        if not documents:
            raise ValueError("파일에서 텍스트를 추출할 수 없습니다. 파일 내용을 확인해주세요.")

        print(f"총 {len(documents)}개 문서/페이지 로드됨")

        # 텍스트 분할
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=["\n\n", "\n", ".", " ", ""]
        )
        splits = text_splitter.split_documents(documents)
        print(f"총 {len(splits)}개 청크로 분할됨")

        # 임베딩 모델 생성
        embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model="text-embedding-3-small")

        # FAISS 벡터 저장소 생성
        print("FAISS 벡터 저장소 생성 중...")
        vectorstore = FAISS.from_documents(
            documents=splits,
            embedding=embeddings
        )

        print("벡터 저장소 생성 완료!")
        return vectorstore

    except Exception as e:
        print(f"이력서 로딩 및 벡터 저장소 생성 중 오류 발생: {str(e)}")
        raise e


In [26]:
# 추천 직무 및 필요 역량 생성 로직
def recommend_job_and_skills(
    vectorstore,
    temperature: float = 0.5,
    num_recommendations: int = 3
) -> Dict:
    """
    주어진 벡터 저장소 (이력서 내용)를 기반으로 추천 직무와 해당 직무에 필요한 역량을 생성합니다.
    생성된 결과는 JSON 형식으로 반환됩니다.
    """
    try:
        # 검색기 설정 (이력서 내용 검색)
        retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 5} # 관련성 높은 5개 청크 검색
        )

        # 프롬프트 템플릿 정의
        template = f"""
        당신은 커리어 컨설턴트이며, 제공된 이력서 내용을 바탕으로 지원자에게 가장 적합한 직무와 해당 직무에 필요한 핵심 역량을 추천해야 합니다.
        응답은 반드시 JSON 형식으로만 반환해야 합니다.

        <이력서 내용>
        {{context}}
        </이력서 내용>

        지침:
        1.  이력서 내용을 면밀히 분석하여 지원자의 경험, 기술 스택, 강점, 관심사 등을 파악하세요.
        2.  지원자의 역량과 경험에 가장 부합하는 {num_recommendations}개의 직무를 추천해주세요.
        3.  각 추천 직무에 대해 해당 직무에서 성공하기 위해 필요한 핵심 역량(기술 및 소프트 스킬)을 3~5가지 나열해주세요.
        4.  전반적인 커리어 개발을 위한 조언을 1~2문장으로 요약하여 추가해주세요.

        응답 JSON 스키마:
        {{{{
        "recommended_roles": [
            {{{{
            "role_name": "추천 직무명 1",
            "required_skills": ["필요 역량 1", "필요 역량 2", "필요 역량 3"]
            }}}},
            {{{{
            "role_name": "추천 직무명 2",
            "required_skills": ["필요 역량 A", "필요 역량 B", "필요 역량 C"]
            }}}}
        ],
        "overall_career_advice": "전반적인 커리어 개발 조언 요약"
        }}}}

        질문: 이 이력서를 가진 지원자에게 어떤 직무와 역량을 추천할까요?

        답변:"""

        prompt = ChatPromptTemplate.from_template(template)

        # LLM 모델 설정
        model = ChatOpenAI(
            model='gpt-4o', 
            temperature=float(temperature),
            api_key=OPENAI_API_KEY
        )

        # JSON 출력을 위한 파서 설정
        parser = JsonOutputParser()

        # RAG 체인 생성
        document_chain = create_stuff_documents_chain(model, prompt)
        rag_chain = create_retrieval_chain(retriever, document_chain)

        # 답변 생성
        response = rag_chain.invoke({'input': "이력서 기반 직무 및 역량 추천을 해주세요."})
        
        # LLM 응답이 JSON 문자열일 경우 파싱
        parsed_response = parser.parse(response['answer'])
        return parsed_response

    except Exception as e:
        print(f"직무 및 역량 추천 중 오류 발생: {str(e)}")
        return {"error": f"직무 및 역량 추천 중 오류가 발생했습니다: {str(e)}"}


In [27]:
#  PDF 파일로 추천 직무 및 필요 역량 생성 - 마케터 버전

resume_file_path = "./resume/예시 파일 (마케터).pdf"

# 파일 존재 여부 확인
if not os.path.exists(resume_file_path):
    print(f"오류: 지정된 경로에 파일이 없습니다: {resume_file_path}")
    print("파일 경로를 확인하거나, 파일을 해당 위치에 넣어주세요.")
else:
    print(f"\n--- '{resume_file_path}' 파일로 추천 직무 및 필요 역량 생성 ---")
    try:
        # 1. PDF 파일 로드 및 벡터 저장소 생성
        vectorstore_marketer = load_resume_to_vector_store(resume_file_path)

        # 2. 추천 직무 및 필요 역량 생성
        recommendations_marketer = recommend_job_and_skills(
            vectorstore_marketer,
            temperature=0.6,
            num_recommendations=3
        )

        # 3. 결과 출력
        if "error" in recommendations_marketer:
            print(f"오류: {recommendations_marketer['error']}")
        else:
            print("\n생성된 추천 직무 및 필요 역량:")
            for i, role in enumerate(recommendations_marketer['recommended_roles']):
                print(f"{i+1}. 직무명: {role['role_name']}")
                print(f"   필요 역량: {', '.join(role['required_skills'])}")
            print(f"\n전반적인 커리어 조언: {recommendations_marketer['overall_career_advice']}")

    except Exception as e:
        print(f"예상치 못한 오류 발생: {e}")


--- './resume/예시 파일 (마케터).pdf' 파일로 추천 직무 및 필요 역량 생성 ---
이력서 파일 로딩 중: ./resume/예시 파일 (마케터).pdf
총 2개 문서/페이지 로드됨
총 4개 청크로 분할됨
FAISS 벡터 저장소 생성 중...
벡터 저장소 생성 완료!

생성된 추천 직무 및 필요 역량:
1. 직무명: 디지털 마케팅 전문가
   필요 역량: 소셜 미디어 콘텐츠 기획 및 제작, 검색 엔진 최적화(SEO) 기초, 데이터 분석 및 성과 측정
2. 직무명: 콘텐츠 마케팅 매니저
   필요 역량: 스토리텔링 및 카피라이팅, 뷰티 트렌드 분석 및 소비자 니즈 파악, 이미지/영상 편집 툴 활용
3. 직무명: 뷰티 산업 전문가
   필요 역량: 최신 뷰티 트렌드에 대한 민감성, 다양한 제품 카테고리에 대한 지식, 경쟁사 및 소비자 행동 분석 능력

전반적인 커리어 조언: 디지털 마케팅과 콘텐츠 기획 경험을 바탕으로 뷰티 산업에서의 전문성을 강화하세요. 최신 트렌드를 지속적으로 학습하고 데이터 기반의 의사결정 능력을 향상시키면 더욱 경쟁력 있는 전문가가 될 수 있습니다.


In [28]:
#  PDF 파일로 추천 직무 및 필요 역량 생성 - 개발자 버전

resume_file_path = "./resume/예시파일 (개발자).pdf"

# 파일 존재 여부 확인
if not os.path.exists(resume_file_path):
    print(f"오류: 지정된 경로에 파일이 없습니다: {resume_file_path}")
    print("파일 경로를 확인하거나, 파일을 해당 위치에 넣어주세요.")
else:
    print(f"\n--- '{resume_file_path}' 파일로 추천 직무 및 필요 역량 생성 ---")
    try:
        # 1. PDF 파일 로드 및 벡터 저장소 생성
        vectorstore_marketer = load_resume_to_vector_store(resume_file_path)

        # 2. 추천 직무 및 필요 역량 생성
        recommendations_marketer = recommend_job_and_skills(
            vectorstore_marketer,
            temperature=0.6,
            num_recommendations=3
        )

        # 3. 결과 출력
        if "error" in recommendations_marketer:
            print(f"오류: {recommendations_marketer['error']}")
        else:
            print("\n생성된 추천 직무 및 필요 역량:")
            for i, role in enumerate(recommendations_marketer['recommended_roles']):
                print(f"{i+1}. 직무명: {role['role_name']}")
                print(f"   필요 역량: {', '.join(role['required_skills'])}")
            print(f"\n전반적인 커리어 조언: {recommendations_marketer['overall_career_advice']}")

    except Exception as e:
        print(f"예상치 못한 오류 발생: {e}")


--- './resume/예시파일 (개발자).pdf' 파일로 추천 직무 및 필요 역량 생성 ---
이력서 파일 로딩 중: ./resume/예시파일 (개발자).pdf
총 3개 문서/페이지 로드됨
총 5개 청크로 분할됨
FAISS 벡터 저장소 생성 중...
벡터 저장소 생성 완료!

생성된 추천 직무 및 필요 역량:
1. 직무명: DevOps Engineer
   필요 역량: Docker & Kubernetes expertise, CI/CD pipeline management, AWS cloud services, Scripting and automation skills, Problem-solving and analytical skills
2. 직무명: Backend Developer
   필요 역량: Java and Spring Boot proficiency, Microservices architecture, Database optimization and management, RESTful API design, Test-driven development
3. 직무명: Cloud Solutions Architect
   필요 역량: AWS architecture design, Microservices and distributed systems, Performance optimization, Security best practices, Stakeholder communication skills

전반적인 커리어 조언: Leverage your strong backend and cloud expertise to pursue roles that focus on scalable, reliable system design. Consider obtaining certifications in AWS and Kubernetes to further solidify your credentials in cloud solutions.


### 사용자의 질문 답변에 대한 피드백 제공

In [1]:
# 이력서 기반 면접 질문 생성 및 답변 피드백 (LangChain)

# 1. 환경 설정 및 라이브러리 임포트
import os
from dotenv import load_dotenv
from typing import List, Dict, Optional

# LangChain 패키지
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.output_parsers import JsonOutputParser # JSON 형식으로 출력 파싱

# 환경 변수에서 API 키 가져오기
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY가 설정되지 않았습니다. .env 파일을 확인해주세요.")

print(f"OPENAI_API_KEY: {OPENAI_API_KEY[:2]}")

OPENAI_API_KEY: sk


In [2]:
# 2. 문서 로딩 및 벡터 저장소 생성 함수
def load_document_to_vector_store(file_path: str, chunk_size: int = 1000, chunk_overlap: int = 100):
    """
    문서 파일을 로드하고 텍스트를 분할하여 FAISS 벡터 저장소를 생성합니다.
    지원 형식: PDF, TXT
    """
    try:
        print(f"문서 파일 로딩 중: {file_path}")

        # 파일 확장자에 따라 적절한 로더 사용
        if file_path.lower().endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        elif file_path.lower().endswith(".txt"):
            loader = TextLoader(file_path)
        else:
            raise ValueError("지원되지 않는 파일 형식입니다. PDF 또는 TXT 파일만 업로드해주세요.")

        documents = loader.load()

        if not documents:
            raise ValueError("파일에서 텍스트를 추출할 수 없습니다. 파일 내용을 확인해주세요.")

        print(f"총 {len(documents)}개 문서/페이지 로드됨")

        # 텍스트 분할
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=["\n\n", "\n", ".", " ", ""]
        )
        splits = text_splitter.split_documents(documents)
        print(f"총 {len(splits)}개 청크로 분할됨")

        # 임베딩 모델 생성
        embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model="text-embedding-3-small")

        # FAISS 벡터 저장소 생성
        print("FAISS 벡터 저장소 생성 중...")
        vectorstore = FAISS.from_documents(
            documents=splits,
            embedding=embeddings
        )

        print("벡터 저장소 생성 완료!")
        return vectorstore

    except Exception as e:
        print(f"문서 로딩 및 벡터 저장소 생성 중 오류 발생: {str(e)}")
        raise e

In [3]:

# 3. 면접 예상 질문 생성 로직
def generate_interview_questions(
    vectorstore,
    company_name: str = "",
    interview_type: str = "general", # 'general', 'technical', 'behavioral'
    desired_job_role: str = "", # 희망 직무 (선택 사항)
    temperature: float = 0.7
) -> Dict:
    """
    주어진 벡터 저장소 (이력서 내용)와 파라미터를 기반으로 면접 예상 질문을 생성합니다.
    생성된 질문은 JSON 형식으로 반환됩니다.
    """
    try:
        # 검색기 설정 (이력서 내용 검색)
        retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 5} # 관련성 높은 5개 청크 검색
        )

        # 질문 유형에 따른 추가 지침
        type_instructions = {
            "general": "지원자의 전반적인 경험, 역량, 회사 적합성 등을 평가할 수 있는 종합적인 질문을 생성해주세요.",
            "technical": "지원자의 이력서에 언급된 기술 스택과 프로젝트 경험을 바탕으로 심도 있는 기술 면접 질문을 생성해주세요.",
            "behavioral": "지원자의 행동 양식, 문제 해결 능력, 팀워크, 리더십 등을 평가할 수 있는 인성 및 행동 기반 면접 질문을 생성해주세요."
        }
        instruction = type_instructions.get(interview_type, type_instructions["general"])

        # 회사 이름이 제공된 경우 추가 지침
        company_instruction = f"면접 볼 회사가 '{company_name}'이라면, 해당 회사의 비전, 제품, 문화와 연관된 질문을 1~2개 포함해주세요." if company_name else ""

        # 희망 직무가 제공된 경우 추가 지침
        job_role_instruction = ""
        if desired_job_role:
            job_role_instruction = f"지원자가 희망하는 직무는 '{desired_job_role}'입니다. 이 직무에 필요한 역량, 경험, 그리고 직무 관련 시나리오에 대한 질문을 1~2개 포함해주세요."
            if interview_type == "technical":
                job_role_instruction += f" 특히 '{desired_job_role}' 직무의 핵심 기술 스택과 관련된 심층 기술 질문을 포함해주세요."
            elif interview_type == "behavioral":
                job_role_instruction += f" 특히 '{desired_job_role}' 직무에서 발생할 수 있는 상황에 대한 행동 기반 질문을 포함해주세요."


        # 프롬프트 템플릿 정의
        # JSON 형식으로 답변을 받기 위한 지시를 포함합니다.
        template = f"""
        당신은 면접관이며, 제공된 이력서 내용을 바탕으로 지원자에게 면접 예상 질문을 생성해야 합니다.
        다음 지침에 따라 질문을 생성하고, 각 질문에 대한 간략한 답변 가이드라인도 함께 제공해주세요.
        응답은 반드시 JSON 형식으로만 반환해야 합니다.

        <이력서 내용>
        {{context}}
        </이력서 내용>

        지침:
        1.  이력서 내용을 면밀히 분석하여 지원자의 경험, 기술 스택, 강점, 약점 등을 파악하세요.
        2.  총 5~7개의 면접 질문을 생성해주세요. (단, 이 데모에서는 첫 번째 질문만 사용합니다.)
        3.  질문 유형: {instruction} {company_instruction} {job_role_instruction}
        4.  각 질문에 대해 답변자가 어떤 점을 강조해야 하는지 1~2문장으로 간략한 '가이드라인'을 포함해주세요.
        5.  전반적인 답변 가이드라인을 1~2문장으로 요약하여 추가해주세요.

        응답 JSON 스키마:
        {{{{
        "questions": [
            {{{{
            "question": "질문 내용 1",
            "guidance": "답변 가이드라인 1"
            }}}},
            {{{{
            "question": "질문 내용 2",
            "guidance": "답변 가이드라인 2"
            }}}}
        ],
        "overall_guidance": "전반적인 답변 가이드라인 요약"
        }}}}

        질문: 이 이력서를 가진 지원자에게 어떤 면접 질문을 할까요?

        답변:"""

        prompt = ChatPromptTemplate.from_template(template)

        # LLM 모델 설정
        model = ChatOpenAI(
            model='gpt-4o', 
            temperature=float(temperature),
            api_key=OPENAI_API_KEY
        )

        # JSON 출력을 위한 파서 설정
        parser = JsonOutputParser()

        # RAG 체인 생성
        document_chain = create_stuff_documents_chain(model, prompt)
        rag_chain = create_retrieval_chain(retriever, document_chain)

        # 답변 생성
        response = rag_chain.invoke({'input': "면접 질문을 생성해주세요."})
        
        # LLM 응답이 JSON 문자열일 경우 파싱
        parsed_response = parser.parse(response['answer'])
        return parsed_response

    except Exception as e:
        print(f"면접 질문 생성 중 오류 발생: {str(e)}")
        return {"error": f"면접 질문 생성 중 오류가 발생했습니다: {str(e)}"}

In [4]:
# 4. 사용자 답변에 대한 피드백 로직 
def critique_user_answer(
    vectorstore, # 이력서/자소서 컨텍스트를 위한 벡터 저장소
    original_question: str,
    user_answer: str,
    company_name: str = "",
    job_role: str = "",
    temperature: float = 0.5
) -> Dict:
    """
    사용자의 답변에 대해 피드백을 제공하고, 개선 방향성을 제시합니다.
    """
    try:
        # 검색기 설정 (이력서/자소서 컨텍스트 검색)
        retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 5} # 관련성 높은 5개 청크 검색
        )

        # 회사 및 직무 정보에 따른 추가 지침
        context_info = ""
        if company_name and job_role:
            context_info = f"지원하는 회사는 '{company_name}'이고, 직무는 '{job_role}'입니다. 이 맥락에서 답변을 평가해주세요."
        elif company_name:
            context_info = f"지원하는 회사는 '{company_name}'입니다. 이 맥락에서 답변을 평가해주세요."
        elif job_role:
            context_info = f"지원하는 직무는 '{job_role}'입니다. 이 맥락에서 답변을 평가해주세요."

        # 프롬프트 템플릿 정의
        template = f"""
        당신은 면접관이자 커리어 코치입니다. 다음 질문과 지원자의 답변을 평가하고, 건설적인 피드백을 JSON 형식으로 제공해주세요.
        평가 시 지원자의 이력서 내용도 참고하여 답변의 적합성과 깊이를 판단해주세요.

        <이력서/자소서 내용>
        {{context}}
        </이력서/자소서 내용>

        <원래 질문>
        {original_question}
        </원래 질문>

        <지원자 답변>
        {user_answer}
        </지원자 답변>

        지침:
        1.  **overall_assessment**: 답변에 대한 전반적인 평가를 1-2문장으로 요약하세요.
        2.  **strengths**: 답변의 주요 강점 2-3가지를 목록으로 제시하세요.
        3.  **areas_for_improvement**: 답변에서 개선이 필요한 부분 2-3가지를 목록으로 제시하세요.
        4.  **actionable_suggestions**: 각 개선점에 대해 구체적이고 실용적인 개선 제안 2-3가지를 목록으로 제시하세요.
        5.  **score_out_of_5**: 답변에 대한 5점 만점 점수를 소수점 첫째 자리까지 매겨주세요. (예: 3.5)
        6.  **next_steps_advice**: 향후 유사한 질문에 답변할 때 도움이 될 만한 전반적인 방향성 조언을 1-2문장으로 제공해주세요.
        7.  {context_info}

        응답 JSON 스키마:
        {{{{
        "overall_assessment": "전반적인 답변에 대한 평가 요약",
        "strengths": [
            "강점 1",
            "강점 2"
        ],
        "areas_for_improvement": [
            "개선 필요 부분 1",
            "개선 필요 부분 2"
        ],
        "actionable_suggestions": [
            "구체적 제안 1",
            "구체적 제안 2"
        ],
        "score_out_of_5": 3.5,
        "next_steps_advice": "향후 답변 방향성 조언"
        }}}}

        질문: 이 답변에 대한 피드백을 제공해주세요.

        답변:"""

        prompt = ChatPromptTemplate.from_template(template)

        # LLM 모델 설정
        model = ChatOpenAI(
            model='gpt-4o', 
            temperature=float(temperature),
            api_key=OPENAI_API_KEY
        )

        # JSON 출력을 위한 파서 설정
        parser = JsonOutputParser()

        # RAG 체인 생성
        document_chain = create_stuff_documents_chain(model, prompt)
        rag_chain = create_retrieval_chain(retriever, document_chain)

        # 답변 생성 (여기서 'input'은 프롬프트 템플릿의 '질문:' 부분에 해당)
        response = rag_chain.invoke({'input': "사용자 답변에 대한 피드백을 생성해주세요."})
        
        # LLM 응답이 JSON 문자열일 경우 파싱
        parsed_response = parser.parse(response['answer'])
        return parsed_response

    except Exception as e:
        print(f"사용자 답변 피드백 중 오류 발생: {str(e)}")
        return {"error": f"사용자 답변 피드백 중 오류가 발생했습니다: {str(e)}"}


In [7]:
# 5. 피드백을 반영한 개선된 답변 생성 로직 
def generate_improved_answer(
    vectorstore,
    original_question: str,
    user_initial_answer: str,
    feedback_results: Dict, # critique_user_answer의 JSON 결과
    company_name: str = "",
    job_role: str = "",
    temperature: float = 0.7
) -> str:
    """
    AI 피드백을 바탕으로 사용자의 답변을 개선하여 새로운 답변을 생성합니다.
    """
    try:
        # 검색기 설정 (이력서/자소서 컨텍스트 검색)
        retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 5} # 관련성 높은 5개 청크 검색
        )

        # 피드백에서 개선 제안 추출
        actionable_suggestions = "\n".join([f"- {s}" for s in feedback_results.get('actionable_suggestions', [])])
        if not actionable_suggestions:
            actionable_suggestions = "제공된 피드백에 구체적인 개선 제안이 없습니다. 답변의 명확성, 간결성, 질문과의 연관성에 중점을 두어 개선해주세요."

        # 회사 및 직무 정보에 따른 추가 지침
        context_info = ""
        if company_name and job_role:
            context_info = f"지원하는 회사는 '{company_name}'이고, 직무는 '{job_role}'입니다. 이 맥락에서 답변을 개선해주세요."
        elif company_name:
            context_info = f"지원하는 회사는 '{company_name}'입니다. 이 맥락에서 답변을 개선해주세요."
        elif job_role:
            context_info = f"지원하는 직무는 '{job_role}'입니다. 이 맥락에서 답변을 개선해주세요."

        # 프롬프트 템플릿 정의
        template = f"""
        당신은 면접 답변 개선 전문가입니다. 다음 면접 질문, 지원자의 초기 답변, 그리고 해당 답변에 대한 AI 피드백을 참고하여,
        가장 효과적이고 설득력 있는 답변으로 개선해주세요.

        <이력서/자소서 내용>
        {{context}}
        </이력서/자소서 내용>

        <원래 질문>
        {original_question}
        </원래 질문>

        <지원자 초기 답변>
        {user_initial_answer}
        </지원자 초기 답변>

        <AI 피드백 (개선 제안 포함)>
        {actionable_suggestions}
        </AI 피드백 (개선 제안 포함)>

        지침:
        1.  AI 피드백의 '구체적인 개선 제안'을 최우선적으로 반영하여 답변을 수정하세요.
        2.  원래 질문에 직접적으로 답변하고, 이력서 내용과 연관된 경험이나 역량을 효과적으로 강조하세요.
        3.  답변은 명확하고 간결하며, 설득력 있게 작성되어야 합니다.
        4.  불필요한 내용은 제거하고, 핵심 메시지를 전달하는 데 집중하세요.
        5.  {context_info}
        6.  개선된 답변만 직접적으로 제공해주세요. 추가적인 설명이나 서론은 필요 없습니다.

        개선된 답변:"""

        prompt = ChatPromptTemplate.from_template(template)

        # LLM 모델 설정
        model = ChatOpenAI(
            model='gpt-4o', 
            temperature=float(temperature),
            api_key=OPENAI_API_KEY
        )

        # RAG 체인 생성 (여기서는 JSON 파서가 필요 없음, 일반 텍스트 출력)
        document_chain = create_stuff_documents_chain(model, prompt)
        rag_chain = create_retrieval_chain(retriever, document_chain)

        # 답변 생성
        response = rag_chain.invoke({'input': "개선된 면접 답변을 작성해주세요."})
        
        return response['answer']

    except Exception as e:
        print(f"개선된 답변 생성 중 오류 발생: {str(e)}")
        return f"개선된 답변 생성 중 오류가 발생했습니다: {str(e)}"


In [ ]:
# 6. 데모 시나리오: 질문 생성 -> 사용자 답변 시뮬레이션 -> 피드백 생성 -> 개선된 답변 생성

resume_file_path = "./resume/예시 파일 (마케터).pdf"

# 파일 존재 여부 확인
if not os.path.exists(resume_file_path):
    print(f"오류: 지정된 경로에 파일이 없습니다: {resume_file_path}")
    print("파일 경로를 확인하거나, 파일을 해당 위치에 넣어주세요.")
else:
    print(f"\n--- '{resume_file_path}' 파일로 면접 질문 생성 및 답변 피드백 시나리오 시작 ---")
    try:
        # 1. 이력서/자소서 파일 로드 및 벡터 저장소 생성
        vectorstore_doc = load_document_to_vector_store(resume_file_path)

        # 2. 면접 예상 질문 생성 (첫 번째 질문만 사용)
        print("\n[단계 1/4] 이력서 기반 면접 질문 생성 중...")
        generated_questions_response = generate_interview_questions(
            vectorstore_doc,
            company_name="ABC 마케팅", # 마케터 이력서에 맞춰 회사명 조정
            interview_type="general",
            desired_job_role="디지털 마케터", # 마케터 이력서에 맞춰 직무 조정
            temperature=0.7
        )

        if "error" in generated_questions_response:
            print(f"질문 생성 오류: {generated_questions_response['error']}")
        elif not generated_questions_response.get('questions'):
            print("생성된 질문이 없습니다.")
        else:
            first_question = generated_questions_response['questions'][0]['question']
            print(f"\n생성된 질문: {first_question}")

            # 3. 사용자 답변 시뮬레이션
            user_simulated_answer = """
            저는 XYZ 에이전시에서 3년간 디지털 마케터로 근무하며 다양한 소셜 미디어 캠페인을 기획하고 실행했습니다.
            특히, 인스타그램 캠페인을 통해 고객 참여율을 20% 향상시킨 경험이 있습니다.
            구글 애널리틱스를 활용하여 캠페인 데이터를 분석하고, 이를 바탕으로 SEO 전략을 최적화하여 웹사이트 유기적 트래픽을 증가시켰습니다.
            이러한 경험을 통해 데이터 기반의 마케팅 전략 수립 및 실행 역량을 강화할 수 있었습니다.
            """
            print(f"\n[단계 2/4] 사용자 답변 (시뮬레이션):")
            print(user_simulated_answer)

            # 4. 사용자 답변에 대한 피드백 생성
            print("\n[단계 3/4] 사용자 답변에 대한 피드백 생성 중...")
            feedback_results = critique_user_answer(
                vectorstore_doc,
                original_question=first_question,
                user_answer=user_simulated_answer,
                company_name="ABC 마케팅",
                job_role="디지털 마케터",
                temperature=0.5
            )

            # 5. 피드백 결과 출력
            if "error" in feedback_results:
                print(f"피드백 생성 오류: {feedback_results['error']}")
            else:
                print("\n--- 사용자 답변에 대한 AI 피드백 ---")
                print(f"전반적인 평가: {feedback_results.get('overall_assessment', 'N/A')}")
                print("\n강점:")
                for i, s in enumerate(feedback_results.get('strengths', [])):
                    print(f"- {s}")
                print("\n개선이 필요한 부분:")
                for i, a in enumerate(feedback_results.get('areas_for_improvement', [])):
                    print(f"- {a}")
                print("\n구체적인 개선 제안:")
                for i, s in enumerate(feedback_results.get('actionable_suggestions', [])):
                    print(f"- {s}")
                print(f"\n점수 (5점 만점): {feedback_results.get('score_out_of_5', 'N/A')}")
                print(f"향후 답변 방향성 조언: {feedback_results.get('next_steps_advice', 'N/A')}")

                # 6. 피드백을 반영한 개선된 답변 생성
                print("\n[단계 4/4] 피드백을 반영한 개선된 답변 생성 중...")
                improved_answer = generate_improved_answer(
                    vectorstore_doc,
                    original_question=first_question,
                    user_initial_answer=user_simulated_answer,
                    feedback_results=feedback_results,
                    company_name="ABC 마케팅",
                    job_role="디지털 마케터",
                    temperature=0.7 # 개선된 답변 생성의 창의성 조절
                )
                print("\n--- AI가 제안하는 개선된 답변 ---")
                print(improved_answer)

    except Exception as e:
        print(f"시나리오 실행 중 예상치 못한 오류 발생: {e}")


--- './resume/예시 파일 (마케터).pdf' 파일로 면접 질문 생성 및 답변 피드백 시나리오 시작 ---
문서 파일 로딩 중: ./resume/예시 파일 (마케터).pdf
총 2개 문서/페이지 로드됨
총 4개 청크로 분할됨
FAISS 벡터 저장소 생성 중...
벡터 저장소 생성 완료!

[단계 1/4] 이력서 기반 면접 질문 생성 중...

생성된 질문: 디지털 마케터로서 뷰티 산업에서의 경험을 통해 얻은 가장 중요한 인사이트는 무엇이었으며, 이를 ABC 마케팅에서 어떻게 활용할 수 있을까요?

[단계 2/4] 사용자 답변 (시뮬레이션):

            저는 XYZ 에이전시에서 3년간 디지털 마케터로 근무하며 다양한 소셜 미디어 캠페인을 기획하고 실행했습니다.
            특히, 인스타그램 캠페인을 통해 고객 참여율을 20% 향상시킨 경험이 있습니다.
            구글 애널리틱스를 활용하여 캠페인 데이터를 분석하고, 이를 바탕으로 SEO 전략을 최적화하여 웹사이트 유기적 트래픽을 증가시켰습니다.
            이러한 경험을 통해 데이터 기반의 마케팅 전략 수립 및 실행 역량을 강화할 수 있었습니다.
            

[단계 3/4] 사용자 답변에 대한 피드백 생성 중...

--- 사용자 답변에 대한 AI 피드백 ---
전반적인 평가: 지원자의 답변은 이전 경험을 잘 설명하고 있지만, 질문에 대한 직접적인 답변과 ABC 마케팅에의 적용 방안이 부족합니다.

강점:
- 이전 직장에서의 구체적인 성과를 제시하여 신뢰성을 높였습니다.
- 데이터 기반의 마케팅 전략 수립과 실행 역량을 강조했습니다.

개선이 필요한 부분:
- 질문에서 요구한 뷰티 산업에서 얻은 인사이트에 대한 직접적인 언급이 부족합니다.
- ABC 마케팅에서의 활용 방안이 구체적으로 제시되지 않았습니다.

구체적인 개선 제안:
- 뷰티 산업에서의 구체적인 인사이트를 제시하고, 이를 통해 어떤 교훈을 얻었는지 설명하세요.
- ABC 마케팅에 해당 인

### 사용자가 직접 답변에 응답하는 형식

In [9]:
# 대화형 면접 질문 및 답변 피드백 (LangChain)

# 환경 설정 및 라이브러리 임포트
import os
from dotenv import load_dotenv
from typing import List, Dict, Optional

# LangChain 패키지
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.output_parsers import JsonOutputParser # JSON 형식으로 출력 파싱

# 환경 변수에서 API 키 가져오기
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY가 설정되지 않았습니다. .env 파일을 확인해주세요.")

print(f"OPENAI_API_KEY: {OPENAI_API_KEY[:2]}")

# 2. 문서 로딩 및 벡터 저장소 생성 함수
def load_document_to_vector_store(file_path: str, chunk_size: int = 1000, chunk_overlap: int = 100):
    """
    문서 파일을 로드하고 텍스트를 분할하여 FAISS 벡터 저장소를 생성합니다.
    지원 형식: PDF, TXT
    """
    try:
        print(f"문서 파일 로딩 중: {file_path}")

        # 파일 확장자에 따라 적절한 로더 사용
        if file_path.lower().endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        elif file_path.lower().endswith(".txt"):
            loader = TextLoader(file_path)
        else:
            raise ValueError("지원되지 않는 파일 형식입니다. PDF 또는 TXT 파일만 업로드해주세요.")

        documents = loader.load()

        if not documents:
            raise ValueError("파일에서 텍스트를 추출할 수 없습니다. 파일 내용을 확인해주세요.")

        print(f"총 {len(documents)}개 문서/페이지 로드됨")

        # 텍스트 분할
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=["\n\n", "\n", ".", " ", ""]
        )
        splits = text_splitter.split_documents(documents)
        print(f"총 {len(splits)}개 청크로 분할됨")

        # 임베딩 모델 생성
        embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model="text-embedding-3-small")

        # FAISS 벡터 저장소 생성
        print("FAISS 벡터 저장소 생성 중...")
        vectorstore = FAISS.from_documents(
            documents=splits,
            embedding=embeddings
        )

        print("벡터 저장소 생성 완료!")
        return vectorstore

    except Exception as e:
        print(f"문서 로딩 및 벡터 저장소 생성 중 오류 발생: {str(e)}")
        raise e


OPENAI_API_KEY: sk


In [ ]:
# 3. 면접 예상 질문 생성 로직 
def generate_interview_questions(
    vectorstore,
    company_name: str = "",
    interview_type: str = "general", # 'general', 'technical', 'behavioral'
    desired_job_role: str = "", # Optional parameter: desired job role
    temperature: float = 0.7
) -> Dict:
    """
    주어진 벡터 저장소 (이력서 내용)와 파라미터를 기반으로 면접 예상 질문을 생성합니다.
    생성된 질문은 JSON 형식으로 반환됩니다.
    """
    try:
        # 검색기 설정 (이력서 내용 검색)
        retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 5} # 관련성 높은 5개 청크 검색
        )

        # 질문 유형에 따른 추가 지침
        type_instructions = {
            "general": "지원자의 전반적인 경험, 역량, 회사 적합성 등을 평가할 수 있는 종합적인 질문을 생성해주세요.",
            "technical": "지원자의 이력서에 언급된 기술 스택과 프로젝트 경험을 바탕으로 심도 있는 기술 면접 질문을 생성해주세요.",
            "behavioral": "지원자의 행동 양식, 문제 해결 능력, 팀워크, 리더십 등을 평가할 수 있는 인성 및 행동 기반 면접 질문을 생성해주세요."
        }
        instruction = type_instructions.get(interview_type, type_instructions["general"])

        # 회사 이름이 제공된 경우 추가 지침
        company_instruction = f"면접 볼 회사가 '{company_name}'이라면, 해당 회사의 비전, 제품, 문화와 연관된 질문을 1~2개 포함해주세요." if company_name else ""

        # 희망 직무가 제공된 경우 추가 지침
        job_role_instruction = ""
        if desired_job_role:
            job_role_instruction = f"지원자가 희망하는 직무는 '{desired_job_role}'입니다. 이 직무에 필요한 역량, 경험, 그리고 직무 관련 시나리오에 대한 질문을 1~2개 포함해주세요."
            if interview_type == "technical":
                job_role_instruction += f" 특히 '{desired_job_role}' 직무의 핵심 기술 스택과 관련된 심층 기술 질문을 포함해주세요."
            elif interview_type == "behavioral":
                job_role_instruction += f" 특히 '{desired_job_role}' 직무에서 발생할 수 있는 상황에 대한 행동 기반 질문을 포함해주세요."


        # 프롬프트 템플릿 정의
        # JSON 형식으로 답변을 받기 위한 지시를 포함합니다.
        template = f"""
        당신은 면접관이며, 제공된 이력서 내용을 바탕으로 지원자에게 면접 예상 질문을 생성해야 합니다.
        다음 지침에 따라 질문을 생성하고, 각 질문에 대한 간략한 답변 가이드라인도 함께 제공해주세요.
        응답은 반드시 JSON 형식으로만 반환해야 합니다.

        <이력서 내용>
        {{context}}
        </이력서 내용>

        지침:
        1.  이력서 내용을 면밀히 분석하여 지원자의 경험, 기술 스택, 강점, 약점 등을 파악하세요.
        2.  총 5~7개의 면접 질문을 생성해주세요. (단, 이 데모에서는 첫 번째 질문만 사용합니다.)
        3.  질문 유형: {instruction} {company_instruction} {job_role_instruction}
        4.  각 질문에 대해 답변자가 어떤 점을 강조해야 하는지 1~2문장으로 간략한 '가이드라인'을 포함해주세요.
        5.  전반적인 답변 가이드라인을 1~2문장으로 요약하여 추가해주세요.

        응답 JSON 스키마:
        {{{{
        "questions": [
            {{{{
            "question": "질문 내용 1",
            "guidance": "답변 가이드라인 1"
            }}}},
            {{{{
            "question": "질문 내용 2",
            "guidance": "답변 가이드라인 2"
            }}}}
        ],
        "overall_guidance": "전반적인 답변 가이드라인 요약"
        }}}}

        질문: 이 이력서를 가진 지원자에게 어떤 면접 질문을 할까요?

        답변:"""

        prompt = ChatPromptTemplate.from_template(template)

        # LLM 모델 설정
        model = ChatOpenAI(
            model='gpt-4o',
            temperature=float(temperature),
            api_key=OPENAI_API_KEY
        )

        # JSON 출력을 위한 파서 설정
        parser = JsonOutputParser()

        # RAG 체인 생성
        document_chain = create_stuff_documents_chain(model, prompt)
        rag_chain = create_retrieval_chain(retriever, document_chain)

        # 답변 생성
        response = rag_chain.invoke({'input': "면접 질문을 생성해주세요."})
        
        # LLM 응답이 JSON 문자열일 경우 파싱
        parsed_response = parser.parse(response['answer'])
        return parsed_response

    except Exception as e:
        print(f"면접 질문 생성 중 오류 발생: {str(e)}")
        return {"error": f"면접 질문 생성 중 오류가 발생했습니다: {str(e)}"}


# 4. 사용자 답변에 대한 피드백 로직
def critique_user_answer(
    vectorstore, # 이력서/자소서 컨텍스트를 위한 벡터 저장소
    original_question: str,
    user_answer: str,
    company_name: str = "",
    job_role: str = "",
    temperature: float = 0.5
) -> Dict:
    """
    사용자의 답변에 대해 피드백을 제공하고, 개선 방향성을 제시합니다.
    """
    try:
        # 검색기 설정 (이력서/자소서 컨텍스트 검색)
        retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 5} # 관련성 높은 5개 청크 검색
        )

        # 회사 및 직무 정보에 따른 추가 지침
        context_info = ""
        if company_name and job_role:
            context_info = f"지원하는 회사는 '{company_name}'이고, 직무는 '{job_role}'입니다. 이 맥락에서 답변을 평가해주세요."
        elif company_name:
            context_info = f"지원하는 회사는 '{company_name}'입니다. 이 맥락에서 답변을 평가해주세요."
        elif job_role:
            context_info = f"지원하는 직무는 '{job_role}'입니다. 이 맥락에서 답변을 평가해주세요."

        # 프롬프트 템플릿 정의
        template = f"""
        당신은 면접관이자 커리어 코치입니다. 다음 질문과 지원자의 답변을 평가하고, 건설적인 피드백을 JSON 형식으로 제공해주세요.
        평가 시 지원자의 이력서 내용도 참고하여 답변의 적합성과 깊이를 판단해주세요.

        <이력서/자소서 내용>
        {{context}}
        </이력서/자소서 내용>

        <원래 질문>
        {original_question}
        </원래 질문>

        <지원자 답변>
        {user_answer}
        </지원자 답변>

        지침:
        1.  **overall_assessment**: 답변에 대한 전반적인 평가를 1-2문장으로 요약하세요.
        2.  **strengths**: 답변의 주요 강점 2-3가지를 목록으로 제시하세요.
        3.  **areas_for_improvement**: 답변에서 개선이 필요한 부분 2-3가지를 목록으로 제시하세요.
        4.  **actionable_suggestions**: 각 개선점에 대해 구체적이고 실용적인 개선 제안 2-3가지를 목록으로 제시하세요.
        5.  **score_out_of_5**: 답변에 대한 5점 만점 점수를 소수점 첫째 자리까지 매겨주세요. (예: 3.5)
        6.  **next_steps_advice**: 향후 유사한 질문에 답변할 때 도움이 될 만한 전반적인 방향성 조언을 1-2문장으로 제공해주세요.
        7.  {context_info}

        응답 JSON 스키마:
        {{{{
        "overall_assessment": "전반적인 답변에 대한 평가 요약",
        "strengths": [
            "강점 1",
            "강점 2"
        ],
        "areas_for_improvement": [
            "개선 필요 부분 1",
            "개선 필요 부분 2"
        ],
        "actionable_suggestions": [
            "구체적 제안 1",
            "구체적 제안 2"
        ],
        "score_out_of_5": 3.5,
        "next_steps_advice": "향후 답변 방향성 조언"
        }}}}

        질문: 이 답변에 대한 피드백을 제공해주세요.

        답변:"""

        prompt = ChatPromptTemplate.from_template(template)

        # LLM 모델 설정
        model = ChatOpenAI(
            model='gpt-4o', 
            temperature=float(temperature),
            api_key=OPENAI_API_KEY
        )

        # JSON 출력을 위한 파서 설정
        parser = JsonOutputParser()

        # RAG 체인 생성
        document_chain = create_stuff_documents_chain(model, prompt)
        rag_chain = create_retrieval_chain(retriever, document_chain)

        # 답변 생성 (여기서 'input'은 프롬프트 템플릿의 '질문:' 부분에 해당)
        response = rag_chain.invoke({'input': "사용자 답변에 대한 피드백을 생성해주세요."})
        
        # LLM 응답이 JSON 문자열일 경우 파싱
        parsed_response = parser.parse(response['answer'])
        return parsed_response

    except Exception as e:
        print(f"사용자 답변 피드백 중 오류 발생: {str(e)}")
        return {"error": f"사용자 답변 피드백 중 오류가 발생했습니다: {str(e)}"}


In [11]:
# 5. 피드백을 반영한 개선된 답변 생성 로직 
def generate_improved_answer(
    vectorstore,
    original_question: str,
    user_initial_answer: str,
    feedback_results: Dict, # critique_user_answer의 JSON 결과
    company_name: str = "",
    job_role: str = "",
    temperature: float = 0.7
) -> str:
    """
    AI 피드백을 바탕으로 사용자의 답변을 개선하여 새로운 답변을 생성합니다.
    """
    try:
        # 검색기 설정 (이력서/자소서 컨텍스트 검색)
        retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 5} # 관련성 높은 5개 청크 검색
        )

        # 피드백에서 개선 제안 추출
        actionable_suggestions = "\n".join([f"- {s}" for s in feedback_results.get('actionable_suggestions', [])])
        if not actionable_suggestions:
            actionable_suggestions = "제공된 피드백에 구체적인 개선 제안이 없습니다. 답변의 명확성, 간결성, 질문과의 연관성에 중점을 두어 개선해주세요."

        # 회사 및 직무 정보에 따른 추가 지침
        context_info = ""
        if company_name and job_role:
            context_info = f"지원하는 회사는 '{company_name}'이고, 직무는 '{job_role}'입니다. 이 맥락에서 답변을 개선해주세요."
        elif company_name:
            context_info = f"지원하는 회사는 '{company_name}'입니다. 이 맥락에서 답변을 개선해주세요."
        elif job_role:
            context_info = f"지원하는 직무는 '{job_role}'입니다. 이 맥락에서 답변을 개선해주세요."

        # 프롬프트 템플릿 정의
        template = f"""
        당신은 면접 답변 개선 전문가입니다. 다음 면접 질문, 지원자의 초기 답변, 그리고 해당 답변에 대한 AI 피드백을 참고하여,
        가장 효과적이고 설득력 있는 답변으로 개선해주세요.

        <이력서/자소서 내용>
        {{context}}
        </이력서/자소서 내용>

        <원래 질문>
        {original_question}
        </원래 질문>

        <지원자 초기 답변>
        {user_initial_answer}
        </지원자 초기 답변>

        <AI 피드백 (개선 제안 포함)>
        {actionable_suggestions}
        </AI 피드백 (개선 제안 포함)>

        지침:
        1.  AI 피드백의 '구체적인 개선 제안'을 최우선적으로 반영하여 답변을 수정하세요.
        2.  원래 질문에 직접적으로 답변하고, 이력서 내용과 연관된 경험이나 역량을 효과적으로 강조하세요.
        3.  답변은 명확하고 간결하며, 설득력 있게 작성되어야 합니다.
        4.  불필요한 내용은 제거하고, 핵심 메시지를 전달하는 데 집중하세요.
        5.  {context_info}
        6.  개선된 답변만 직접적으로 제공해주세요. 추가적인 설명이나 서론은 필요 없습니다.

        개선된 답변:"""

        prompt = ChatPromptTemplate.from_template(template)

        # LLM 모델 설정
        model = ChatOpenAI(
            model='gpt-4o', 
            temperature=float(temperature),
            api_key=OPENAI_API_KEY
        )


        document_chain = create_stuff_documents_chain(model, prompt)
        rag_chain = create_retrieval_chain(retriever, document_chain)

        response = rag_chain.invoke({'input': "개선된 면접 답변을 작성해주세요."})
        
        return response['answer']

    except Exception as e:
        print(f"개선된 답변 생성 중 오류 발생: {str(e)}")
        return f"개선된 답변 생성 중 오류가 발생했습니다: {str(e)}"


In [16]:
# 6. 데모 시나리오: 질문 생성 -> 사용자 입력 답변 -> 피드백 생성 -> 개선된 답변 생성

resume_file_path = "./resume/예시 파일 (마케터).pdf" 

# # 'data' 디렉토리가 없으면 생성합니다.
# os.makedirs("./data", exist_ok=True)

# if not os.path.exists(resume_file_path):
#     try:
#         from reportlab.lib.pagesizes import letter
#         from reportlab.pdfgen import canvas
#         c = canvas.Canvas(resume_file_path, pagesize=letter)
#         c.drawString(100, 750, "이름: 이마케")
#         c.drawString(100, 730, "연락처: 010-1111-2222")
#         c.drawString(100, 710, "이메일: lee.marketer@example.com")
#         c.drawString(100, 690, "경력: XYZ 에이전시 (2021-현재) - 디지털 마케터")
#         c.drawString(100, 670, "  - 소셜 미디어 캠페인 기획 및 실행 (성과 20% 향상)")
#         c.drawString(100, 650, "  - 구글 애널리틱스 활용 데이터 분석 및 보고서 작성")
#         c.drawString(100, 630, "  - SEO 최적화 및 콘텐츠 마케팅 전략 수립")
#         c.drawString(100, 610, "기술: SEO, SEM, Google Analytics, SNS 마케팅, 콘텐츠 마케팅, 데이터 분석")
#         c.drawString(100, 590, "학력: 마케팅 대학교 마케팅학과")
#         c.save()
#         print(f"'{resume_file_path}' 파일이 생성되었습니다.")
#     except ImportError:
#         print("reportlab 모듈이 설치되지 않아 PDF 파일을 생성할 수 없습니다. 'pip install reportlab' 명령어로 설치하거나, 실제 PDF 파일을 사용해주세요.")
#     except Exception as e:
#         print(f"PDF 파일 생성 중 오류 발생: {e}")


# 파일 존재 여부 확인
if not os.path.exists(resume_file_path):
    print(f"오류: 지정된 경로에 파일이 없습니다: {resume_file_path}")
    print("파일 경로를 확인하거나, 파일을 해당 위치에 넣어주세요.")
else:
    print(f"\n--- '{resume_file_path}' 파일로 면접 질문 생성 및 답변 피드백 시나리오 시작 ---")
    try:
        # 1. 이력서/자소서 파일 로드 및 벡터 저장소 생성
        vectorstore_doc = load_document_to_vector_store(resume_file_path)

        # 2. 면접 예상 질문 생성 (첫 번째 질문만 사용)
        print("\n[단계 1/4] 이력서 기반 면접 질문 생성 중...")
        generated_questions_response = generate_interview_questions(
            vectorstore_doc,
            company_name="ABC 마케팅", # 마케터 이력서에 맞춰 회사명 조정
            interview_type="general",
            desired_job_role="디지털 마케터", # 마케터 이력서에 맞춰 직무 조정
            temperature=0.7
        )

        if "error" in generated_questions_response:
            print(f"질문 생성 오류: {generated_questions_response['error']}")
        elif not generated_questions_response.get('questions'):
            print("생성된 질문이 없습니다.")
        else:
            first_question = generated_questions_response['questions'][0]['question']
            print(f"\n생성된 질문: {first_question}")

            # 3. 사용자 답변 입력 받기
            print("\n[단계 2/4] 위 질문에 대한 답변을 입력해주세요 (입력 후 Enter):")
            user_input_answer = input("당신의 답변: ") # 사용자 입력 받기
            print(f"당신이 입력한 답변:\n{user_input_answer}") # 사용자 입력 답변 출력

            # 4. 사용자 답변에 대한 피드백 생성
            print("\n[단계 3/4] 사용자 답변에 대한 피드백 생성 중...")
            feedback_results = critique_user_answer(
                vectorstore_doc,
                original_question=first_question,
                user_answer=user_input_answer, # 사용자 입력 답변 사용
                company_name="ABC 마케팅",
                job_role="디지털 마케터",
                temperature=0.5
            )

            # 5. 피드백 결과 출력
            if "error" in feedback_results:
                print(f"피드백 생성 오류: {feedback_results['error']}")
            else:
                print("\n--- 사용자 답변에 대한 AI 피드백 ---")
                print(f"전반적인 평가: {feedback_results.get('overall_assessment', 'N/A')}")
                print("\n강점:")
                for i, s in enumerate(feedback_results.get('strengths', [])):
                    print(f"- {s}")
                print("\n개선이 필요한 부분:")
                for i, a in enumerate(feedback_results.get('areas_for_improvement', [])):
                    print(f"- {a}")
                print("\n구체적인 개선 제안:")
                for i, s in enumerate(feedback_results.get('actionable_suggestions', [])):
                    print(f"- {s}")
                print(f"\n점수 (5점 만점): {feedback_results.get('score_out_of_5', 'N/A')}")
                print(f"향후 답변 방향성 조언: {feedback_results.get('next_steps_advice', 'N/A')}")

                # 6. 피드백을 반영한 개선된 답변 생성
                print("\n[단계 4/4] 피드백을 반영한 개선된 답변 생성 중...")
                improved_answer = generate_improved_answer(
                    vectorstore_doc,
                    original_question=first_question,
                    user_initial_answer=user_input_answer, # 사용자 입력 답변 사용
                    feedback_results=feedback_results,
                    company_name="ABC 마케팅",
                    job_role="디지털 마케터",
                    temperature=0.7 # 개선된 답변 생성의 창의성 조절
                )
                print("\n--- AI가 제안하는 개선된 답변 ---")
                print(improved_answer)

    except Exception as e:
        print(f"시나리오 실행 중 예상치 못한 오류 발생: {e}")

# 생성된 임시 파일 삭제 (선택 사항)
# if os.path.exists(resume_file_path):
#     os.remove(resume_file_path)
#     print(f"'{resume_file_path}' 파일이 삭제되었습니다.")


--- './resume/예시 파일 (마케터).pdf' 파일로 면접 질문 생성 및 답변 피드백 시나리오 시작 ---
문서 파일 로딩 중: ./resume/예시 파일 (마케터).pdf
총 2개 문서/페이지 로드됨
총 4개 청크로 분할됨
FAISS 벡터 저장소 생성 중...
벡터 저장소 생성 완료!

[단계 1/4] 이력서 기반 면접 질문 생성 중...

생성된 질문: ABC 마케팅에서 진행하는 디지털 마케팅 캠페인에 기여할 수 있는 당신의 가장 큰 강점은 무엇이라고 생각하십니까?

[단계 2/4] 위 질문에 대한 답변을 입력해주세요 (입력 후 Enter):
당신이 입력한 답변:
ABC 마케팅의 디지털 마케팅 캠페인에 기여할 수 있는 저의 가장 큰 강점은 데이터 기반의 문제 해결 능력과 지속적인 학습 및 적용 능력이라고 생각합니다.  이전 디지털 마케팅 실전 스터디에서 퍼포먼스 마케팅 케이스 스터디를 통해 가설 수립 및 검증의 중요성을 깊이 이해했습니다. 단순히 데이터를 보고 현상을 파악하는 것을 넘어, '왜 이런 결과가 나왔을까?'라는 질문을 던지고, '어떤 가설을 세워야 이 문제를 해결할 수 있을까?'를 고민하며 구체적인 검증 방안을 수립하는 데 익숙합니다.  실제 웹사이트 개편 프로젝트에서는 이러한 가설 수립 및 검증 프로세스를 적용하여 웹사이트의 복잡한 내비게이션, 제품 상세 페이지 정보 부족, 모바일 최적화 미흡 등의 문제점을 데이터 분석, 사용자 설문조사 및 인터뷰, 경쟁사 분석을 통해 명확히 파악하고 개선 방향을 도출했습니다. 이 경험을 통해 저는 문제의 본질을 파악하고, 데이터를 활용하여 논리적인 해결책을 제시하며, 실제 성과로 연결시키는 데 강점이 있음을 확인했습니다.

[단계 3/4] 사용자 답변에 대한 피드백 생성 중...

--- 사용자 답변에 대한 AI 피드백 ---
전반적인 평가: 지원자는 데이터 기반 문제 해결 능력과 학습 적용 능력을 강조하며, 실제 경험을 통해 이를 뒷받침하는 사례를 잘 제시했습니다.

강점:
- 데이터 분석을 통한

### 자소서 파일을 받아서 자소서 컨설팅 진행 (미리 칸만 만들어두고 나중에 상황보고 추가할지 말지 정하기)